This notebook takes the csv and computes the reading grade level of the text report and writes it to the csv in a column called grade.

In [ ]:
import pandas as pd 
from fkscore import fkscore

In [ ]:
data = 'Resources/current_data.csv'
df = pd.read_csv(data)

In [ ]:
df['grade'] = ''

In [ ]:
for index,row in df.iterrows():
    try:
        text = str(df.loc[index,'text'])
        f=fkscore(text)
        df.loc[index,'grade'] = f.score['grade']
    except ZeroDivisionError:
        df.loc[index,'grade'] = 'Unknown'

In [ ]:
df = df[['text','date_time','city','state','shape','duration','grade']]
df.head()

In [ ]:
df.to_csv('Resources/dont_touch_me.csv')